Market Basket Analysis (MBA)

Tujuan dari MBA adalah untuk mengetahui produk-produk mana yang mungkin akan dibeli secara bersamaan.

In [1]:
#Mengimpor library pandas & numpy
import pandas as pd
import numpy as np

In [2]:
#Membaca file
dataset_transaksi = pd.read_csv('https://storage.googleapis.com/dqlab-dataset/transaksi_dqlab_retail.tsv', sep='\t')

#Menampilkan isi data
print(dataset_transaksi)

      Kode Transaksi              Nama Barang
0                 #1                     Kaos
1                 #1             Shampo Biasa
2                 #1      Sepatu Sport merk Z
3                 #1            Serum Vitamin
4                 #1  Baju Renang Pria Dewasa
...              ...                      ...
33663          #3450     Flat Shoes Ballerina
33664          #3450       Sepatu Sandal Anak
33665          #3450      Sepatu Kulit Casual
33666          #3450     Celana Jogger Casual
33667          #3450      Sepatu Sport merk Z

[33668 rows x 2 columns]


Selanjutnya, untuk memastikan lagi tipe data dan tidak ada data yang null. Aku menggunakan function info() untuk melihat detail dari dataset tersebut.

In [3]:
#Melihat detail tipe data
dataset_transaksi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33668 entries, 0 to 33667
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Kode Transaksi  33668 non-null  object
 1   Nama Barang     33668 non-null  object
dtypes: object(2)
memory usage: 526.2+ KB


Aku menyadari barang yang terdapat pada data transaksi retail belum dikelompokkan berdasarkan kode transaksi. Melihat permasalahan itu, aku merasa perlu mengelompokkan barang sesuai **kode transaksi yang sama**. Hal ini perlu dilakukan, untuk mengetahui barang apa saja yang terdapat pada basket yang sama.

Aku membutuhkan satu kolom bantuan diberi nama “Flag”. Kolom “Flag” ini akan membantu menandakan barang apa saja yang terdapat pada satu keranjang yang sama.

Setelah itu, aku membuat kode berikut untuk menambahkan kolom baru bernama “Flag” dengan value 1.

In [4]:
#Flag digunakan untuk menandakan barang/item terdapat pada basket.
dataset_transaksi['Flag'] = 1

#Menampilkan isi data
print(dataset_transaksi)

      Kode Transaksi              Nama Barang  Flag
0                 #1                     Kaos     1
1                 #1             Shampo Biasa     1
2                 #1      Sepatu Sport merk Z     1
3                 #1            Serum Vitamin     1
4                 #1  Baju Renang Pria Dewasa     1
...              ...                      ...   ...
33663          #3450     Flat Shoes Ballerina     1
33664          #3450       Sepatu Sandal Anak     1
33665          #3450      Sepatu Kulit Casual     1
33666          #3450     Celana Jogger Casual     1
33667          #3450      Sepatu Sport merk Z     1

[33668 rows x 3 columns]


Kemudian, aku mengelompokkan “Flag” berdasarkan kolom “Kode Transaksi” dan “Nama Barang” menggunakan function groupby()

In [5]:
#Melakukan pengelompokkan Flag berdasarkan kolom Kode Transaksi dan Nama Barang
basket = dataset_transaksi.groupby(['Kode Transaksi','Nama Barang'])['Flag'].sum().unstack().reset_index().fillna(0).set_index('Kode Transaksi')

#Menampilkan basket
print(basket)

Nama Barang     Atasan Baju Belang  Atasan Kaos Putih  Baju Batik Wanita  \
Kode Transaksi                                                             
#1                             0.0                0.0                0.0   
#10                            0.0                0.0                1.0   
#100                           0.0                0.0                0.0   
#1000                          0.0                0.0                1.0   
#1001                          0.0                0.0                0.0   
...                            ...                ...                ...   
#995                           0.0                1.0                0.0   
#996                           0.0                0.0                0.0   
#997                           0.0                0.0                1.0   
#998                           0.0                0.0                1.0   
#999                           0.0                0.0                1.0   

Nama Barang

Karena tujuan memberikan “Flag” adalah hanya untuk mengetahui barang apa yang terdapat dalam satu keranjang, maka perlu melakukan normalisasi pada data.

Caranya dengan menandai bahwa barang yang ada di keranjang nilainya adalah 1, sedangkan barang yang tidak ada dalam keranjang nilainya 0.

In [6]:
#Membuat function untuk menormalisasi data
def encode_units(x):
	if x <= 0 :
		return 0
	if x > 0:
		return 1

Kemudian function tersebut diterapkan pada dataset.Aku membuat dataset baru bernama basket_encode untuk membedakan dengan dataset sebelumnya.

In [7]:
#Menerapkan fungsi encode_units pada dataset
basket_encode = basket.applymap(encode_units)

#Menampilkan basket_encode
print(basket_encode)

Nama Barang     Atasan Baju Belang  Atasan Kaos Putih  Baju Batik Wanita  \
Kode Transaksi                                                             
#1                               0                  0                  0   
#10                              0                  0                  1   
#100                             0                  0                  0   
#1000                            0                  0                  1   
#1001                            0                  0                  0   
...                            ...                ...                ...   
#995                             0                  1                  0   
#996                             0                  0                  0   
#997                             0                  0                  1   
#998                             0                  0                  1   
#999                             0                  0                  1   

Nama Barang

<ipython-input-7-87c3ef2e7479>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encode = basket.applymap(encode_units)


Mengenal Lift dan Confidence

**Confidence** adalah ukuran persentase seberapa kuat hubungan antar item dalam aturan asosiasi.
Contoh: pada suatu market, produk Teh dan Gula memiliki nilai confidence 50%, artinya pelanggan yang membeli Teh punya kemungkinan sebesar 50% juga membeli Gula.

**Lift** digunakan untuk memvalidasi aturan asosiasi, dengan mengukur ketepatan support dan confidence.
Kombinasi itemset dinyatakan valid dan kuat jika nilai Lift > 1.
- Jika hasil menunjukkan nilai < 1, maka memiliki korelasi negatif.
- Jika hasil menunjukkan nilai > 1, maka memiliki korelasi positif.
- Jika hasil menunjukkan nilai = 1, maka tidak ada korelasi.

Mengenal Algoritma Apriori

Salah satu algoritma yang dapat digunakan dalam Market Basket Analysis adalah algoritma Apriori.

Algoritma Apriori adalah algoritma yang dapat digunakan untuk menentukan frequent itemset untuk aturan asosiasi boolean. Tujuan algoritma ini, untuk mencari kombinasi item yang memenuhi syarat minimum dari nilai support dalam basis data.

Frequent itemset, menunjukkan itemset yang memiliki frekuensi kemunculan lebih dari nilai minimum yang ditentukan. Misalkan nilai minimum 2, maka semua itemsets yang frekuensi kemunculannya lebih dari atau sama dengan 2 kali disebut frequent.

Nilai support dari sebuah item diperoleh dari jumlah transaksi mengandung item tersebut dibandingkan total transaksi.

In [8]:
#Mengimport algoritma apriori
from mlxtend.frequent_patterns import apriori

In [9]:
#Menerapkan algoritma apriori untuk mendapatkan frequent_itemset
frequent_itemset = apriori(basket_encode, min_support=0.03, use_colnames=True)

print(frequent_itemset)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


       support                                           itemsets
0     0.118841                                (Atasan Kaos Putih)
1     0.380290                                (Baju Batik Wanita)
2     0.121159                 (Baju Kaos Anak - Karakter Kartun)
3     0.115942                     (Baju Kaos Anak - Superheroes)
4     0.089275                               (Baju Kaos Olahraga)
...        ...                                                ...
1255  0.031304  (Tas Tangan, Tank Top, Celana Pendek Jeans, Ba...
1256  0.042609  (Baju Kemeja Putih, Serum Vitamin, Shampo Bias...
1257  0.038551  (Tank Top, Tas Travel, Wedges Hitam, Gembok Ko...
1258  0.047536  (Sepatu Sekolah Hitam W, Blouse Denim, Serum V...
1259  0.037391  (Atasan Kaos Putih, Tank Top, Tas Travel, Wedg...

[1260 rows x 2 columns]


Setelah mendapatkan frequent_itemset dan menentukan aturan asosiasi. Langkah pertama yang aku lakukan adalah import association_rules dengan kode berikut.

In [10]:
#Mengimport association_rules
from mlxtend.frequent_patterns import association_rules

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
#Menerapkan association_rules berdasarkan frequent_itemset
rules = association_rules(frequent_itemset, metric='lift', min_threshold=1).sort_values('lift', ascending=False).reset_index(drop=True)

print(rules)

                                            antecedents  \
0     (Baju Kaos Anak - Karakter Kartun, Tank Top, C...   
1                             (Tas Tangan, Cover Koper)   
2                                (Tas Tangan, Tank Top)   
3     (Cover Koper, Baju Kaos Anak - Karakter Kartun...   
4     (Cover Koper, Baju Kaos Anak - Karakter Kartun...   
...                                                 ...   
5143                               (Tali Pinggang Anak)   
5144                     (Tas Waist Bag, Serum Vitamin)   
5145                                (Baju Kemeja Putih)   
5146                        (Serum Vitamin, Hair Dryer)   
5147                        (Tali Pinggang Gesper Pria)   

                                            consequents  antecedent support  \
0                             (Tas Tangan, Cover Koper)            0.034493   
1     (Baju Kaos Anak - Karakter Kartun, Tank Top, C...            0.031884   
2     (Cover Koper, Baju Kaos Anak - Karakter Kartun..

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


berdasarkan hasil analisis yang kudapatkan barang yang memiliki asosiasi paling berat adalah cover koper dan tas tangan. Oleh karena itu, jika cover koper dan tas tangan diletakkan berdekatan, ada kemungkinan orang yang ingin membeli cover koper akan ikut membeli tas tangan begitu pula sebaliknya.

